In [1]:
class Graph:
    def __init__(self, nodes_in_graph=[]):
        self.nodes_in_graph = nodes_in_graph
        
    def add_node(self, node):
        self.nodes_in_graph.append(node)
    
    def add_nodes(self, node_list):
        for node in node_list:
            self.add_node(node)
            
    def get_node(self, nodename):
        for node in self.nodes_in_graph:
            if node.nodename == nodename:
                return node
            
    def build_cost_tables(self):
        for node in self.nodes_in_graph:
            node.build_cost_table(self)
            
    def get_shortest_path(self, from_nodename, to_nodename):
        from_node = self.get_node(from_nodename)
        return from_node.get_shortest_path_to(to_nodename)
    
class Node:
    def __init__(self, nodename, connections):
        self.nodename = nodename
        self.connections = connections
        
        self.cost_table = {self.nodename: {'cost': 0,
                                           'previous': None}}
        
        for name, cost in self.connections.items():
            self.cost_table[name] = {'cost': cost,
                                     'previous': self.nodename}
    
    def __repr__(self):
        cost_table_header = '\tNode\tCost\tPrevious\n'
        cost_table_str = ''.join(
            [f'\t{node}\t{vals["cost"]}\t{vals["previous"]}\n' 
             for node, vals
             in self.cost_table.items()]
        )
        
        return f'nodename" {self.nodename}\nconnections: {self.connections}\ncost_table:\n {cost_table_header}{cost_table_str}'

    def __str__(self):
        cost_table_header = '\tNode\tCost\tPrevious\n'
        cost_table_str = ''.join(
            [f'\t{node}\t{vals["cost"]}\t{vals["previous"]}\n' 
             for node, vals
             in self.cost_table.items()]
        )
        
        return f'nodename" {self.nodename}\nconnections: {self.connections}\ncost_table:\n {cost_table_header}{cost_table_str}'


    def get_shortest_path_to(self, some_node_name):
        shortest_path = []
        costs = []
        total_cost = self.cost_table[some_node_name]['cost']
        
        current_node = some_node_name[:]
        
        for _ in range(100):
            shortest_path.append(current_node)
            
            if self.cost_table[current_node]['previous'] != None:
                costs.append(self.cost_table[current_node]['cost']
                             - self.cost_table[
                                 self.cost_table[current_node]['previous']
                             ]['cost'])
                
            current_node = self.cost_table[current_node]['previous']
            
            if current_node == None:
                break
        
        costs = list(reversed([str(cost) for cost in costs]))
        
        return_str = ''
        
        for idx, let in enumerate(reversed(shortest_path)):
            if idx < len(costs):
                return_str += let + f' -{costs[idx]}-> '
            else:
                return_str += let
                
        return_str += f'\ntotal cost: {total_cost}'
        
        return return_str
        
    def build_cost_table(self, graph):
        '''
        dijkstras
        
        Let cost from this node to itself = 0
        Populate cost table entries w/ all nodes in graph
        Let distance to all other nodes = 10**9 (e.g., infinity)
        
        Repeat:
            Visit unvisited node w/ smallest known dist from this start node
                For current node, examine its unvisted neighbors
                For current node, calc dist of each neighbor from this start node
                If calc'd distance of a node is less than current dist in cost table, update the shortest dist in cost table
                Update prev node for each updated dist
                Add current node to list of visited nodes and remove from list of unvisited nods
        Until all nodes are visited        
        '''
        visited = [self.nodename]
        
        # populate cost table entries w/ remaining nodes in graph
        # let distance to all other nodes be 10**9 (standin for infinity)
        for node in graph.nodes_in_graph:
            if node.nodename not in self.cost_table:
                self.cost_table[node.nodename] = {'cost': 10**9, 'previous': None}
                
        # poupulate unvisited list w/ all nodenames
        unvisited = [node.nodename
                     for node
                     in graph.nodes_in_graph
                     if node.nodename != self.nodename]
        
        # instead of using while loop, assume a max of N nodes in unvisited and create exit cond
        # step thru the nodes prioritizing lowest cost nodes and updating cost table accordingly
        for _ in range(100):
            # search for node w/ lowest cost
            min_cost, min_node = min([(self.cost_table[nodename]['cost'], nodename) for nodename in unvisited])
            
            min_node = graph.get_node(min_node)
            
            for nodename, cost in min_node.connections.items():
                dist_from_start = min_cost + cost
                
                if dist_from_start < self.cost_table[nodename]['cost']:
                    self.cost_table[nodename]['cost'] = dist_from_start
                    self.cost_table[nodename]['previous'] = min_node.nodename
            
                visited.append(min_node.nodename)
            
                if min_node.nodename in unvisited:
                    unvisited.remove(min_node.nodename)
                
            if len(unvisited) == 0:
                break

In [2]:
# define nodes
A = Node('A', {'B': 5, 'C': 7, 'D': 2})
B = Node('B', {'A': 5, 'E': 4})
C = Node('C', {'A': 7, 'D': 3, 'F': 5})
D = Node('D', {'A': 2, 'C': 3, 'E': 4, 'G': 6})
E = Node('E', {'B': 4, 'D': 4, 'G': 2})
F = Node('F', {'C': 5, 'G': 7, 'H': 6})
G = Node('G', {'D': 6, 'E': 2, 'F': 7, 'H': 3})
H = Node('H', {'F': 6, 'G': 3})

# A = Node('A', {'B': 4, 'C': 3})
# B = Node('B', {'A': 4, 'C': 2, 'D': 3})
# C = Node('C', {'A': 3, 'B': 2, 'E': 4})
# D = Node('D', {'B': 3, 'E': 5})
# E = Node('E', {'C': 4, 'D': 5})

graph = Graph()
graph.add_nodes([A, B, C, D, E, F, G, H])
# graph.add_nodes([A, B, C, D, E])
graph.build_cost_tables()
print(graph.get_shortest_path('A', 'H'))
# print(graph.get_shortest_path('A', 'E'))

A -2-> D -6-> G -3-> H
total cost: 11
